In [84]:
import numpy as np
from collections import defaultdict
import pandas as pd
from scipy import stats
import urllib
import urllib.parse
import pathlib
import os

## Dataframe is 323 rows by 63 columns with missing values present throughout
- Data should be loaded into JSON format to extract hierarchical relationships

In [86]:
'''--------------------- Part One: Dataframe construction -----------------------'''
df0 = pd.read_csv(r"C:\Users\David\Documents\MacBook Backup\Python files\RGMP\summer 2019\my_dash_app\myenv\env_with_financial-1.csv",header=None)
df1 = df0.replace(np.NaN,'')
df = np.array(df1)
df1

,0,1,2,3,4,5,6,7,8,9,...,54,55,56,57,58,59,60,61,62,63
0,,,Damages,,,,,,,,...,,,,,,,,,,
1,,,Damages Bucket,,,,,,,,...,,,,Volatility Bucket,Company Valutation Bucket,,,,Descriptive Bucket,
2,,,Environmental Fines (Amount),,Total Discharges to water,,Amount of Spills (Tonnes),,Hazardous Waste (Tonnes),,...,,Debt to Equity Ratio,,Adjusted Beta,Market Cap,,EV,,Industry,Country
3,Ticker,Name,Average annual change [5y],% Change [5y],Average annual change [5y],% Change [5y],Average annual change [5y],% Change [5y],Average annual change [5y],% Change [5y],...,2017 Value,Average annual change [5y],2017 Value,2017 Value,Average annual change [5y],2017 Value,Average annual change [5y],2017 Value,2017 Value,2017 Value
4,GRO CN,GrowMax Resources Corp,,,,,,,,,...,,-,-,0.87,-12729249.42,21036300,-63926097.11,-12449700,Materials,CA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
318,D-U CN,Dream Office Real Estate Investment Trus,,,,,,,,,...,19.35,-1.1,78.8,0.29,-8788637.22,1711293200,-704381876.9,2981983200,Real Estate,CA
319,CPX CN,Capital Power Corp,,,-4790400,-0.123936666,,,0,,...,19.26,2.07,70.64,1.8,10851010.88,2575808300,193344876.7,5493808300,Utilities,CA
320,CHR CN,Chorus Aviation Inc,,,,,,,,,...,6.82,24.57,397.7,-0.12,27662696.58,1286581700,300580466.1,2330184700,Industrials,CA
321,CR CN,Crew Energy Inc,,,,,,,,,...,66.87,3.16,36.3,8.67,-17263564.85,469210900,-103582622.7,785049900,Energy,CA


In [125]:
# We can see that there are 4 rows comprising headings for the hierarchical dataset
# This function will extract each level of heading for a given datapoint and add it to a dictionary
# x is the index value  for each non-empty value in the dataset
def get_dict_keys(x, df):
    return [[k.strip() for idx, k in enumerate(df[i]) if k != '' and idx <= x][-1] for i in range(4)]

def get_json():

    # create dict of company data
    nested_dict = lambda: defaultdict(nested_dict)
    data = nested_dict()

    for idx_row, row in enumerate(df):
        for idx_val, value in enumerate(row):
            # passing first four headers and first two columns
            if idx_row < 4 or idx_val < 2:
                continue

            # if data exists
            if value != '' and value != '--' and value != '-':

                # ticker
                tick = df[:, 0][idx_row]

                # find keys and add value
                keys_fin = get_dict_keys(idx_val, df)
                data[tick][keys_fin[0]][keys_fin[1]][keys_fin[2]][keys_fin[3]] = value

    return dict(data)

In [126]:
# get all keys at every depth
def key_at_depth(dct, dpt):
     if dpt > 0:
         return [key for k, subdct in dct.items() for key in key_at_depth(subdct, dpt-1)]
     else:
         return dct.keys()

# Retrieve data array given a set of keys
def get_data_arr(data, key1, key2, key3, key4):
    tickers, data_points = [], []
    for t, _ in data.items():
        try:
            data_points.append(float(data[t][key1][key2][key3][key4]))
            tickers.append(t)
        except:
            pass

    return tickers, data_points


data = get_json()

## Initialize Arrays

In [127]:

# get all keys at every depth
keys_fin = [list(set(key_at_depth(data, i))) for i in range(5)]
keys_fin = [sorted(set(x)) for x in keys_fin]

#creating bubble plot dropdown menu options
'''damages topic'''
damages_q = keys_fin[1][0]
dmgs_bckts_list = [damages_q]

damages_bucket_q = keys_fin[2][1]
env_fines_amt_q = keys_fin[3][9]
discharges_water_q = keys_fin[3][19]
amt_spills_q = keys_fin[3][2]
hazardous_waste_q = keys_fin[3][12]
dmgs_bckt_list = [env_fines_amt_q, discharges_water_q, amt_spills_q, hazardous_waste_q]

'''emissions topic'''
emissions_q = keys_fin[1][1]

total_emissions_bucket_q = keys_fin[2][8]
direct_co2_emissions_q = keys_fin[3][5]
ghg_emissions_q = keys_fin[3][22]
ods_emissions_q = keys_fin[3][15]
tot_emns_bckt_list = [direct_co2_emissions_q, ghg_emissions_q, ods_emissions_q]

emissions_per_sales_energy_q = keys_fin[2][3]
ghg_intensity_energy_q = keys_fin[3][11]
ghg_intensity_sales_q = keys_fin[3][10]
emns_sales_eng_list = [ghg_intensity_energy_q, ghg_intensity_sales_q]

emns_bckts_list = [total_emissions_bucket_q, emissions_per_sales_energy_q]

'''recycled resources topic'''
recycled_resources_q = keys_fin[1][3]

water_recycled_bucket_q = keys_fin[2][13]
pct_water_recycled_q = keys_fin[3][0]
tot_water_recycled_q = keys_fin[3][25]
wat_recy_bckt_list = [pct_water_recycled_q, tot_water_recycled_q]

energy_recycled_bucket_q = keys_fin[2][5]
renewables_pct_energy_q = keys_fin[3][23]
renewable_energy_use_q = keys_fin[3][17]
eng_recy_bckt_list = [renewables_pct_energy_q,renewable_energy_use_q]

waste_recycled_bucket_q = keys_fin[2][11]
waste_recycled_pct_total_q = keys_fin[3][28]
waste_recycled_q = keys_fin[3][27]
waste_recy_bckt_list = [waste_recycled_pct_total_q, waste_recycled_q]

recy_res_bckts_list = [water_recycled_bucket_q,energy_recycled_bucket_q,waste_recycled_bucket_q]

'''resource consumption topic'''
resource_consumption_q = keys_fin[1][4]

water_consumption_bucket_q = keys_fin[2][12]
water_use_q = keys_fin[3][26]
water_intensity_sales_q = keys_fin[3][29]
wat_cons_bckt_list = [water_use_q, water_intensity_sales_q]

energy_consumption_bucket_q = keys_fin[2][4]
energy_consumption_q = keys_fin[3][20]
energy_intensity_sales_q = keys_fin[3][8]
eng_cons_bckt_list = [energy_consumption_q, energy_intensity_sales_q]

waste_consumption_bucket_q = keys_fin[2][10]
tot_waste_q = keys_fin[3][24]
waste_generated_sales_q = keys_fin[3][30]
wast_cons_bckt_list = [tot_waste_q, waste_generated_sales_q]

rsc_cons_bckts_list = [water_consumption_bucket_q, energy_consumption_bucket_q, waste_consumption_bucket_q]

'''Financials topic'''
financials_topic_q = keys_fin[1][2]

gaap_bucket_q = keys_fin[2][6]
revenue_q = keys_fin[3][18]
eps_q = keys_fin[3][6]
gaap_bckt_list = [revenue_q, eps_q]

ratios_bucket_q = keys_fin[2][7]
pe_ratio_q = keys_fin[3][16]
debt_to_equity_q = keys_fin[3][4]
ratios_bckt_list = [pe_ratio_q, debt_to_equity_q]

volatility_bucket_q = keys_fin[2][9]
adjusted_beta_q = keys_fin[3][1]
vol_bckt_list = [adjusted_beta_q]

company_valuation_bucket_q = keys_fin[2][0]
market_cap_q = keys_fin[3][14]
ev_q = keys_fin[3][7]
comp_val_bckt_list = [market_cap_q, ev_q]

descriptive_bucket_q = keys_fin[2][2]
industry_q = keys_fin[3][13]
country_q = keys_fin[3][3]
desc_bckt_list = [industry_q, country_q]

finc_bckts_list = [gaap_bucket_q, ratios_bucket_q, volatility_bucket_q,\
                   company_valuation_bucket_q, descriptive_bucket_q ]
'''calculations'''
pct_change_q = keys_fin[4][0]
value_2017_q = keys_fin[4][1]
value_change_q = keys_fin[4][2]
prop_change_q = keys_fin[4][3]

topics_list = [damages_q, emissions_q, recycled_resources_q, resource_consumption_q]

buckets_list = [damages_bucket_q, total_emissions_bucket_q, emissions_per_sales_energy_q,\
                water_recycled_bucket_q, energy_recycled_bucket_q, waste_recycled_bucket_q,\
                water_consumption_bucket_q, energy_consumption_bucket_q, waste_consumption_bucket_q]

metrics_list = [env_fines_amt_q, discharges_water_q, amt_spills_q, hazardous_waste_q,\
                direct_co2_emissions_q, ghg_emissions_q, ods_emissions_q,\
                ghg_intensity_energy_q, ghg_intensity_sales_q, pct_water_recycled_q,\
                tot_water_recycled_q, renewables_pct_energy_q,renewable_energy_use_q,\
                waste_recycled_pct_total_q, waste_recycled_q,water_use_q,water_intensity_sales_q,\
                energy_consumption_q, energy_intensity_sales_q,tot_waste_q, waste_generated_sales_q]

size_parameters_list = [market_cap_q,revenue_q]

calculations_list = [pct_change_q, value_2017_q, value_change_q, prop_change_q]


lst = [env_fines_amt_q, discharges_water_q, amt_spills_q, hazardous_waste_q]
base_key1 = keys_fin[1][3]
base_key2 = keys_fin[2][5]
base_key3 = keys_fin[3][17]
base_key4 = keys_fin[4][0]
base_x,base_y = get_data_arr(data, base_key1, base_key2, base_key3, base_key4)

In [128]:
data

{'GRO CN': defaultdict(<function __main__.get_json.<locals>.<lambda>()>,
             {'Financials': defaultdict(<function __main__.get_json.<locals>.<lambda>()>,
                          {'GAAP Highlights Bucket': defaultdict(<function __main__.get_json.<locals>.<lambda>()>,
                                       {'Revenue': defaultdict(<function __main__.get_json.<locals>.<lambda>()>,
                                                    {'Average annual change [5y]': '-9241574.6'}),
                                        'EPS': defaultdict(<function __main__.get_json.<locals>.<lambda>()>,
                                                    {'Average annual change [5y]': '0.01',
                                                     '2017 Value': '-0.02'})}),
                           'Volatility Bucket': defaultdict(<function __main__.get_json.<locals>.<lambda>()>,
                                       {'Adjusted Beta': defaultdict(<function __main__.get_json.<locals>.<lambda>()>,
 

# Breakdown of Categories

## Emissions
emns_bckts_list  = [total_emissions_bucket_q, emissions_per_sales_energy_q]

- tot_emns_bckt_list = [direct_co2_emissions_q, ghg_emissions_q, ods_emissions_q]
- emns_sales_eng_list = [ghg_intensity_energy_q, ghg_intensity_sales_q]

## Damages
Damages = 'Damages Bucket'

- dmgs_bckt_list = [env_fines_amt_q, discharges_water_q, amt_spills_q, hazardous_waste_q]

## Recycled Resources
recy_res_bckts_list = [water_recycled_bucket_q,energy_recycled_bucket_q,waste_recycled_bucket_q]

- wat_recy_bckt_list = [pct_water_recycled_q, tot_water_recycled_q]
- eng_recy_bckt_list = [renewables_pct_energy_q,renewable_energy_use_q]
- waste_recy_bckt_list = [waste_recycled_pct_total_q, waste_recycled_q]


## Resource Consumption
rsc_cons_bckts_list = [water_consumption_bucket_q, energy_consumption_bucket_q, waste_consumption_bucket_q]

- wat_cons_bckt_list = [water_use_q, water_intensity_sales_q]
- eng_cons_bckt_list = [energy_consumption_q, energy_intensity_sales_q]
- wast_cons_bckt_list = [tot_waste_q, waste_generated_sales_q]


## Financials
finc_bckts_list = [gaap_bucket_q, ratios_bucket_q, volatility_bucket_q,company_valuation_bucket_q, descriptive_bucket_q ]

- gaap_bckt_list = [revenue_q, eps_q]
- ratios_bckt_list = [pe_ratio_q, debt_to_equity_q]
- vol_bckt_list = [adjusted_beta_q]
- comp_val_bckt_list = [market_cap_q, ev_q]
- desc_bckt_list = [industry_q, country_q]

